In [1]:
from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
from nuscenes.utils import splits
import mmcv
import numpy as np
import pprint
import argparse
import os
import torch
import logging
from path import Path
from utils import custom_transform
from dataset.KITTI_dataset import KITTI
from dataset.NuScenes_dataset import NuScenes_Dataset
from model import DeepVIO
from collections import defaultdict
from utils.kitti_eval import KITTI_tester, data_partition
import numpy as np
import math
import os
import glob
import numpy as np
import time
import scipy.io as sio
import torch
from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import math
from utils.utils import *
import nuscenes.utils.geometry_utils as geometry_utils
from pyquaternion import Quaternion
from utils.utils import rotationError, read_pose_from_text
from collections import Counter
from scipy.ndimage import gaussian_filter1d
from scipy.signal.windows import triang
from scipy.ndimage import convolve1d
from torch.utils.data import Dataset
from utils import custom_transform
from utils.nuscenes_eval import NuScenes_Tester

In [2]:
#########################################################################################
dataroot = '/data/public/360_3D_OD_Dataset/nuscenes'
cam_names = ["CAM_FRONT", "CAM_FRONT_RIGHT", "CAM_BACK_RIGHT", "CAM_BACK", "CAM_BACK_LEFT", "CAM_FRONT_LEFT"]
# ckpt_path = './pretrained_models/vf_512_if_256_3e-05.model'
ckpt_path = './pretrained_models/best_0.10.pth'
#########################################################################################

In [3]:
nusc_can = NuScenesCanBus(dataroot=dataroot)
nusc = NuScenes(version='v1.0-trainval', dataroot=dataroot, verbose=False)

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "7"  # Set the GPUs 2 and 3 to use
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = '7'
# device = 'cuda:2'

---

In [65]:
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# parser.add_argument('--img_w', type=int, default=512, help='image width')
# parser.add_argument('--img_h', type=int, default=256, help='image height')
parser.add_argument('--img_w', type=int, default=448, help='image width')
parser.add_argument('--img_h', type=int, default=256, help='image height')
parser.add_argument('--seq_len', type=int, default=11, help='sequence length for LSTM')
parser.add_argument('--keyframe_only', default=False, action='store_true', help='whether to use keyframe only')
parser.add_argument('--sampling_rate', type=int, default=1, help='sampling rate for the training input data')
parser.add_argument('--max_imu_length', type=int, default=10, help='maximum imu length for each sequence') # 50 for nuscenes, 11 for kitti



parser.add_argument('--save_dir', type=str, default='./results', help='path to save the result')
parser.add_argument('--seed', type=int, default=0, help='random seed')
parser.add_argument('--v_f_len', type=int, default=512, help='visual feature length')
parser.add_argument('--i_f_len', type=int, default=256, help='imu feature length')
parser.add_argument('--fuse_method', type=str, default='cat', help='fusion method [cat, soft, hard]')
parser.add_argument('--imu_dropout', type=float, default=0, help='dropout for the IMU encoder')
parser.add_argument('--rnn_hidden_size', type=int, default=1024, help='size of the LSTM latent')
parser.add_argument('--rnn_dropout_out', type=float, default=0.2, help='dropout for the LSTM output layer')
parser.add_argument('--rnn_dropout_between', type=float, default=0.2, help='dropout within LSTM')
parser.add_argument('--weight_decay', type=float, default=5e-6, help='weight decay for the optimizer')
parser.add_argument('--workers', type=int, default=4, help='number of workers')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs_warmup', type=int, default=5, help='number of epochs for warmup')
parser.add_argument('--epochs_joint', type=int, default=15, help='number of epochs for joint training')
parser.add_argument('--epochs_fine', type=int, default=5, help='number of epochs for finetuning')
parser.add_argument('--lr_warmup', type=float, default=5e-4, help='learning rate for warming up stage')
parser.add_argument('--lr_joint', type=float, default=5e-5, help='learning rate for joint training stage')
parser.add_argument('--lr_fine', type=float, default=1e-6, help='learning rate for finetuning stage')
parser.add_argument('--eta', type=float, default=0.05, help='exponential decay factor for temperature')
parser.add_argument('--temp_init', type=float, default=5, help='initial temperature for gumbel-softmax')
parser.add_argument('--Lambda', type=float, default=3e-5, help='penalty factor for the visual encoder usage')
parser.add_argument('--experiment_name', type=str, default='experiment', help='experiment name')
parser.add_argument('--optimizer', type=str, default='Adam', help='type of optimizer [Adam, SGD]')
parser.add_argument('--pretrain_flownet',type=str, default='./pretrained_models/flownets_bn_EPE2.459.pth.tar', help='wehther to use the pre-trained flownet')
parser.add_argument('--pretrain', type=str, default=None, help='path to the pretrained model')
parser.add_argument('--hflip', default=False, action='store_true', help='whether to use horizonal flipping as augmentation')
parser.add_argument('--color', default=False, action='store_true', help='whether to use color augmentations')
parser.add_argument('--print_frequency', type=int, default=10, help='print frequency for loss values')
parser.add_argument('--weighted', default=False, action='store_true', help='whether to use weighted sum')

args = parser.parse_args(args=[])

In [66]:
mmcv.mkdir_or_exist(args.save_dir)
checkpoints_dir = os.path.join(args.save_dir, "experiment_1")
mmcv.mkdir_or_exist(checkpoints_dir)

# Create logs
logger = logging.getLogger(args.experiment_name)
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger.info('----------------------------------------TRAINING----------------------------------')
logger.info('PARAMETER ...')
logger.info(args)

# Load the dataset
transform_train = [custom_transform.ToTensor(), custom_transform.Resize((args.img_h, args.img_w))]
if args.hflip:
    transform_train += [custom_transform.RandomHorizontalFlip()]
if args.color:
    transform_train += [custom_transform.RandomColorAug()]
transform_train = custom_transform.Compose(transform_train)

val_dataset = NuScenes_Dataset(dataroot,
                                mode='val',
                                sequence_length=args.seq_len,
                                keyframe_only=args.keyframe_only,
                                sampling_rate=args.sampling_rate,
                                max_imu_length=args.max_imu_length,
                                cam_names=cam_names,
                                transform=transform_train,
                                nusc=nusc,
                                nusc_can=nusc_can,
                                args=args)

val_scene_datasets = val_dataset.get_val_dataset()

skipped scenes: 73
total samples: 1740


In [67]:
# GPU selections
str_ids = device.split(',')
gpu_ids = []
for str_id in str_ids:
    id = int(str_id)
    if id >= 0:
        gpu_ids.append(id)
if len(gpu_ids) > 0:
    torch.cuda.set_device(gpu_ids[0])

# Initialize the tester
tester = NuScenes_Tester(val_scene_datasets)

# Model initialization
model = DeepVIO(args)

model.load_state_dict(torch.load(ckpt_path))
print('load model %s'%ckpt_path)

# Feed model to GPU
model.cuda(gpu_ids[0])
model = torch.nn.DataParallel(model, device_ids = gpu_ids)

model.eval()
errors = tester.eval(model, 'gumbel-softmax', num_gpu=len(gpu_ids))

# results_dir = os.path.join("./vis", "experiment_20231031_nonkeyframe_ep50")
# mmcv.mkdir_or_exist(results_dir)
# tester.generate_plots(results_dir, 30)

load model ./pretrained_models/best_0.10.pth
testing sequence 0


100%|██████████| 23/23 [00:10<00:00,  2.19it/s]


testing sequence 1


100%|██████████| 22/22 [00:10<00:00,  2.12it/s]


testing sequence 2


...

100%|██████████| 23/23 [00:10<00:00,  2.13it/s]


testing sequence 75


100%|██████████| 23/23 [00:10<00:00,  2.17it/s]


testing sequence 76


100%|██████████| 23/23 [00:10<00:00,  2.13it/s]


In [68]:
# average errors and usages for each camera
cam_errors = {}
for idx, scene_error in enumerate(errors):
    cam_name = cam_names[idx % len(cam_names)]
    
    if cam_name not in cam_errors:
        cam_errors[cam_name] = []
    cam_errors[cam_name].append(scene_error)
    
for cam_name in cam_errors:
    # get the average error and usage
    t_rmse_sum = 0
    r_rmse_sum = 0
    usage_sum = 0
    for scene_error in cam_errors[cam_name]:
        t_rmse_sum += scene_error['t_rmse']
        r_rmse_sum += scene_error['r_rmse']
        usage_sum += scene_error['usage']
    t_rmse_avg = t_rmse_sum / len(cam_errors[cam_name])
    r_rmse_avg = r_rmse_sum / len(cam_errors[cam_name])
    usage_avg = usage_sum / len(cam_errors[cam_name])
    print('cam_name: %s, t_rmse_avg: %f, r_rmse_avg: %f, usage_avg: %f'%(cam_name, t_rmse_avg, r_rmse_avg, usage_avg))

cam_name: CAM_FRONT, t_rmse_avg: 0.085961, r_rmse_avg: 0.131352, usage_avg: 74.866221
cam_name: CAM_FRONT_RIGHT, t_rmse_avg: 0.100964, r_rmse_avg: 0.128089, usage_avg: 77.926421
cam_name: CAM_BACK_RIGHT, t_rmse_avg: 0.109078, r_rmse_avg: 0.149532, usage_avg: 73.098206
cam_name: CAM_BACK, t_rmse_avg: 0.089556, r_rmse_avg: 0.134369, usage_avg: 80.836482
cam_name: CAM_BACK_LEFT, t_rmse_avg: 0.111413, r_rmse_avg: 0.164639, usage_avg: 71.332001
cam_name: CAM_FRONT_LEFT, t_rmse_avg: 0.104699, r_rmse_avg: 0.162785, usage_avg: 74.468051


In [ ]:
# seq len: 11

# cam_name: CAM_FRONT, t_rmse_avg: 0.085961, r_rmse_avg: 0.131352, usage_avg: 74.866221
# cam_name: CAM_FRONT_RIGHT, t_rmse_avg: 0.100964, r_rmse_avg: 0.128089, usage_avg: 77.926421
# cam_name: CAM_BACK_RIGHT, t_rmse_avg: 0.109078, r_rmse_avg: 0.149532, usage_avg: 73.098206
# cam_name: CAM_BACK, t_rmse_avg: 0.089556, r_rmse_avg: 0.134369, usage_avg: 80.836482
# cam_name: CAM_BACK_LEFT, t_rmse_avg: 0.111413, r_rmse_avg: 0.164639, usage_avg: 71.332001
# cam_name: CAM_FRONT_LEFT, t_rmse_avg: 0.104699, r_rmse_avg: 0.162785, usage_avg: 74.468051

In [ ]:
# sequence length: 8

# cam_name: CAM_FRONT, t_rmse_avg: 0.090500, r_rmse_avg: 0.133517, usage_avg: 70.971491
# cam_name: CAM_FRONT_RIGHT, t_rmse_avg: 0.102885, r_rmse_avg: 0.128351, usage_avg: 74.554342
# cam_name: CAM_BACK_RIGHT, t_rmse_avg: 0.117192, r_rmse_avg: 0.151300, usage_avg: 68.218005
# cam_name: CAM_BACK, t_rmse_avg: 0.099378, r_rmse_avg: 0.135416, usage_avg: 75.728807
# cam_name: CAM_BACK_LEFT, t_rmse_avg: 0.119768, r_rmse_avg: 0.166766, usage_avg: 67.228638
# cam_name: CAM_FRONT_LEFT, t_rmse_avg: 0.111839, r_rmse_avg: 0.163894, usage_avg: 69.925339

In [ ]:
# sequence length: 5

# cam_name: CAM_FRONT, t_rmse_avg: 0.100449, r_rmse_avg: 0.140109, usage_avg: 63.155962
# cam_name: CAM_FRONT_RIGHT, t_rmse_avg: 0.107117, r_rmse_avg: 0.123524, usage_avg: 70.018399
# cam_name: CAM_BACK_RIGHT, t_rmse_avg: 0.126840, r_rmse_avg: 0.153187, usage_avg: 60.018217
# cam_name: CAM_BACK, t_rmse_avg: 0.106605, r_rmse_avg: 0.140015, usage_avg: 69.032102
# cam_name: CAM_BACK_LEFT, t_rmse_avg: 0.120381, r_rmse_avg: 0.172458, usage_avg: 61.706168
# cam_name: CAM_FRONT_LEFT, t_rmse_avg: 0.118807, r_rmse_avg: 0.168503, usage_avg: 62.812478